In [1]:
import csv
from difflib import SequenceMatcher

#findDOMAINfromURL

from urllib.parse import urlparse

# url from front, bring the domain from url
def extract_domain_from_url(url):
    parsed_url = urlparse(url)
    return parsed_url.netloc

# Example usage: main 
url = "https://chat.openai.com/c/c55df84f-bdbb-4a59-947f-6630c9d2371e"
domain = extract_domain_from_url(url)
print("Domain:", domain)




Domain: chat.openai.com


In [2]:
#findURLfromCSV

# Function to calculate Jaccard similarity between two sets
def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0.0


# Function to create shingles from a given string
def create_shingles(input_str, k=3):
    shingles = set()
    for i in range(len(input_str) - k + 1):
        shingle = input_str[i:i + k]
        shingles.add(shingle)
    return shingles


# Function to calculate similarity using SequenceMatcher from difflib
def similarity_using_difflib(str1, str2):
    matcher = SequenceMatcher(None, str1, str2)
    return matcher.ratio()



# Path to your CSV file
csv_file_path = "Cites.csv"

# Given URL to find the most similar one
given_url = extract_domain_from_url(url)

# Hyperparameters
threshold = 0.9

# Read URLs from the CSV file
with open(csv_file_path, 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        if row:  # Ensure the row is not empty
            csv_url = row[0]

            # Calculate similarity using SequenceMatcher
            similarity = similarity_using_difflib(given_url, csv_url)

            # If similarity is above a certain threshold, consider it a match
            if similarity > threshold:
                print(f"Similarity with {csv_url}: {similarity}")
                


Similarity with chat.openai.com  : 0.9375


In [3]:
# getCertificatefromRealsite
    
import ssl
import socket
from datetime import datetime
from OpenSSL import crypto


def get_certificate_info(domain, port=443):
    try:
        context = ssl.create_default_context()
        with context.wrap_socket(socket.create_connection((domain, port)), server_hostname=domain) as ssl_socket:
            # Fetch the SSL/TLS certificate
            der_cert = ssl_socket.getpeercert(True)

            # Load the certificate using OpenSSL.crypto
            try:
                x509 = crypto.load_certificate(crypto.FILETYPE_ASN1, der_cert)
            except crypto.Error as cert_error:
                return f"Error loading certificate: {cert_error}"

            # Extract certificate information
            certificate_info = {
                'Domain': domain,
                'Subject': dict(x509.get_subject().get_components()),
                'Issuer': dict(x509.get_issuer().get_components()),
                'Serial Number': x509.get_serial_number(),
                'Version': x509.get_version(),
                'Not Before': datetime.strptime(x509.get_notBefore().decode(), '%Y%m%d%H%M%SZ'),
                'Not After': datetime.strptime(x509.get_notAfter().decode(), '%Y%m%d%H%M%SZ'),
                'Signature Algorithm': x509.get_signature_algorithm(),
                'Public Key': crypto.dump_publickey(crypto.FILETYPE_PEM, x509.get_pubkey()).decode(),
            }

            return certificate_info

    except Exception as e:
        return f"Error: {e}"


# Example usage
if similarity > threshold:
    domain_name = extract_domain_from_url(url)
    certificate_info = get_certificate_info(domain_name)
else:
    print ("There is no similiar domain to this site")

if isinstance(certificate_info, dict):
    print("SSL/TLS Certificate Information:")
    for key, value in certificate_info.items():
        print(f"{key}: {value}")

    # Save entire certificate info to text file
    cert_txt_filename = f"{domain_name}_certificate_info.txt"
    with open(cert_txt_filename, 'w') as cert_txtfile:
        for key, value in certificate_info.items():
            cert_txtfile.write(f"{key}: {value}\n")

    print(f"Certificate information saved to {cert_txt_filename}")


SSL/TLS Certificate Information:
Domain: chat.openai.com
Subject: {b'C': b'US', b'ST': b'California', b'L': b'San Francisco', b'O': b'Cloudflare, Inc.', b'CN': b'chat.openai.com'}
Issuer: {b'C': b'US', b'O': b'Cloudflare, Inc.', b'CN': b'Cloudflare Inc ECC CA-3'}
Serial Number: 18534752069071642456640483092012639886
Version: 2
Not Before: 2023-10-25 00:00:00
Not After: 2024-10-23 23:59:59
Signature Algorithm: b'ecdsa-with-SHA256'
Public Key: -----BEGIN PUBLIC KEY-----
MFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAELYia1YcrVMZDdt21jKAVP00D1hWw
JbX3Pd2SERdzLyUqr8CW8p/n9YO2Bdgsszekb1XX7CgWfmZfs08DfDcPMQ==
-----END PUBLIC KEY-----

Certificate information saved to chat.openai.com_certificate_info.txt


In [4]:
import ssl
import socket
from OpenSSL import crypto
from datetime import datetime
from urllib.parse import urlparse

def asn1_to_datetime(asn1_time):
    # Convert ASN.1 time to datetime
    return datetime.strptime(asn1_time.decode(), "%Y%m%d%H%M%SZ")

def get_certificate_and_public_key(url):
    try:
        parsed_url = urlparse(url)
        host = parsed_url.hostname

        if not host:
            return "Error: Unable to extract hostname from the URL"

        # Establish a connection to the server
        context = ssl.create_default_context()
        with socket.create_connection((host, 443)) as sock:
            with context.wrap_socket(sock, server_hostname=host) as ssock:
                # Retrieve the SSL/TLS certificate
                cert_data = ssock.getpeercert(True)

                # Load the certificate
                x509 = crypto.load_certificate(crypto.FILETYPE_ASN1, cert_data)

                # Extract certificate details
                domain = host
                subject = dict(x509.get_subject().get_components())
                issuer = dict(x509.get_issuer().get_components())
                serial_number = x509.get_serial_number()
                version = x509.get_version()
                not_before = asn1_to_datetime(x509.get_notBefore())
                not_after = asn1_to_datetime(x509.get_notAfter())
                signature_algorithm = x509.get_signature_algorithm().decode()

                # Get the public key
                pubkey = x509.get_pubkey()

                # Format the certificate and public key as PEM-encoded strings
                certificate_str = f"Domain: {domain}\n" \
                                  f"Subject: {subject}\n" \
                                  f"Issuer: {issuer}\n" \
                                  f"Serial Number: {serial_number}\n" \
                                  f"Version: {version}\n" \
                                  f"Not Before: {not_before}\n" \
                                  f"Not After: {not_after}\n" \
                                  f"Signature Algorithm: {signature_algorithm}\n" \
                                  f"Public Key: {crypto.dump_publickey(crypto.FILETYPE_PEM, pubkey).decode()}"

                # Save the certificate to a file
                with open("certificate.txt", "w") as cert_file:
                    cert_file.write(certificate_str)

                return "Certificate saved to certificate.txt"
    except Exception as e:
        return f"Error: {e}"

# Call the function with the URL outside the function definition
result = get_certificate_and_public_key(url)
print(result)


Certificate saved to certificate.txt


In [5]:
!pip install datasketch

from datasketch import MinHash

def create_minhash_signature(text, num_perm=128):
    minhash = MinHash(num_perm=num_perm)
    for word in text:
        minhash.update(word.encode('utf-8'))
    return minhash

def compute_jaccard_similarity(minhash1, minhash2):
    return minhash1.jaccard(minhash2)

def compare_text_files(file1_path, file2_path):
    # Read and preprocess text from files
    text1 = preprocess_text(read_file(file1_path))
    text2 = preprocess_text(read_file(file2_path))

    # Create MinHash signatures
    minhash1 = create_minhash_signature(text1)
    minhash2 = create_minhash_signature(text2)

    # Compute Jaccard similarity
    similarity = compute_jaccard_similarity(minhash1, minhash2)

    return similarity

def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def preprocess_text(text):
    # Add your text preprocessing steps here
    # Tokenization, removing stop words, etc.
    return tokenize_and_remove_stopwords(text)

def tokenize_and_remove_stopwords(text):
    # Add your tokenization and stop word removal logic here
    # This can be done using libraries like NLTK or spaCy
    # For simplicity, you can split the text into words here
    return text.split()

# Example usage
file1_path = 'chat.openai.com_certificate_info.txt'
file2_path = 'certificate.txt'
similarity = compare_text_files(file1_path, file2_path)
print(f"Jaccard Similarity: {similarity}")


Jaccard Similarity: 0.9765625
